# Find and filter the Campaigns 100% junk sources in the YM report

In [ ]:
from pathlib import Path
import csv
from collections import defaultdict

In [ ]:
NORM_TIME_SEC=30

In [ ]:
def make_from_string(val):
    """
    Transforms value and return. If empty returns None.
    """
    
    if val=="Не определено":
        return None
    
    try:
        return float(val)   
    except (TypeError, ValueError):
        pass
    
    try:
        h, m, s = map(int, val.split(':'))
        return h*3600 + m*60 + s
    except (TypeError, ValueError):
        pass
    
    return val

Extract and transform raw data: Add csv from Yandex Metrics

In [ ]:
p=Path('sources.csv')

with p.open(encoding='utf-8-sig') as f:
    reader=csv.DictReader(f)
    raw_rows=[row for row in reader]
    raw_rows=raw_rows[1:]
    
raw_rows

In [ ]:
norm_rows=[]
for dic in raw_rows:
    norm_vals={k: make_from_string(v) for k, v in dic.items()}
    norm_vals["Total goals"]=sum((
            norm_vals['Целевые посетители (Переход в Telegram)'],
            norm_vals['Целевые посетители (Переход в WhatsApp)'],
            norm_vals['Целевые посетители (Автоцель: клик по email)'],
            norm_vals['Целевые посетители (Отправка формы (go to thank-you-page))'],
            norm_vals['Целевые посетители (Клик кнопки Звонок)'],
            norm_vals['Целевые посетители (Клик номера телефона (footer))']
    ))
    norm_rows.append(norm_vals)
    
norm_rows

Filter all the junk rows (but not all sources are junk)

In [ ]:
junk_rows=[dic for dic in norm_rows if dic['Total goals']==0 and dic['Время на сайте']<NORM_TIME_SEC and dic['Площадка']]
good_rows=[dic for dic in norm_rows if (dic['Total goals']!=0 or dic['Время на сайте']>=NORM_TIME_SEC) and dic['Площадка']]

junk_rows

In [ ]:
junk_rows_sources=set(map(lambda x: x['Площадка'], junk_rows))
good_rows_sources=set(map(lambda x: x['Площадка'], good_rows))

junk_rows_sources

find 100% junk sources

In [ ]:
totally_junk_sources=[source for source in junk_rows_sources if source not in good_rows_sources]

totally_junk_sources

In [ ]:
len(totally_junk_sources)

Preparation for aggregation

In [ ]:
updated_norm_rows=[]
for dic in norm_rows:
    dic['Junk source']= 'junk' if dic['Площадка'] in totally_junk_sources else 'ok'
    dic['Triple source']= (dic['Источник трафика'], dic['Площадка'], dic['Страна'])
    updated_norm_rows.append(dic)
    
updated_norm_rows

In [ ]:
pre_table_agg=defaultdict(lambda: {
  'Источник трафика': '',
  'Площадка': '',
  'Страна': '',
  'Визиты': 0.0,
  'Посетители': 0.0,
  'Коэффициент отказов': 0.0,
  'Среднее время на сайте, сек': 0,
  'Целевые посетители (Переход в Telegram)': 0.0,
  'Целевые посетители (Переход в WhatsApp)': 0.0,
  'Целевые посетители (Автоцель: клик по email)': 0.0,
  'Целевые посетители (Отправка формы (go to thank-you-page))': 0.0,
  'Целевые посетители (Клик кнопки Звонок)': 0.0,
  'Целевые посетители (Клик номера телефона (footer))': 0.0,
  'Total goals': 0.0,
  'Junk source': '',
})

for dic in updated_norm_rows:
  pre_table_agg[dic['Triple source']]['Источник трафика'] = dic['Источник трафика']
  pre_table_agg[dic['Triple source']]['Площадка'] = dic['Площадка']
  pre_table_agg[dic['Triple source']]['Страна'] = dic['Страна']
  pre_table_agg[dic['Triple source']]['Визиты'] += dic['Визиты']
  pre_table_agg[dic['Triple source']]['Посетители'] += dic['Посетители']
  pre_table_agg[dic['Triple source']]['Коэффициент отказов'] += dic['Отказы']*dic['Визиты']
  pre_table_agg[dic['Triple source']]['Среднее время на сайте, сек'] += dic['Время на сайте']*dic['Визиты']
  pre_table_agg[dic['Triple source']]['Целевые посетители (Переход в Telegram)'] += dic['Целевые посетители (Переход в Telegram)']
  pre_table_agg[dic['Triple source']]['Целевые посетители (Переход в WhatsApp)'] += dic['Целевые посетители (Переход в WhatsApp)']
  pre_table_agg[dic['Triple source']]['Целевые посетители (Автоцель: клик по email)'] += dic['Целевые посетители (Автоцель: клик по email)']
  pre_table_agg[dic['Triple source']]['Целевые посетители (Отправка формы (go to thank-you-page))'] += dic['Целевые посетители (Отправка формы (go to thank-you-page))']
  pre_table_agg[dic['Triple source']]['Целевые посетители (Клик кнопки Звонок)'] += dic['Целевые посетители (Клик кнопки Звонок)']
  pre_table_agg[dic['Triple source']]['Целевые посетители (Клик номера телефона (footer))'] += dic['Целевые посетители (Клик номера телефона (footer))']
  pre_table_agg[dic['Triple source']]['Total goals'] += dic['Total goals']
  pre_table_agg[dic['Triple source']]['Junk source'] = dic['Junk source']
  
pre_table_agg
    

    

Aggregate and write to file

In [ ]:
pre_table_agg_2 = pre_table_agg.values()

full_table_agg=[]
for row in pre_table_agg_2:

    row['Коэффициент отказов'] = row['Коэффициент отказов']/row['Визиты']
    row['Среднее время на сайте, сек'] = row['Среднее время на сайте, сек']/row['Визиты']

    full_table_agg.append(row)
    
       
full_table_agg

In [ ]:
pw=Path('output_full_table.csv')

with pw.open('w', encoding='utf-8-sig', newline='') as f:
    fieldnames=full_table_agg[0].keys()
    
    writer=csv.DictWriter(f, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    writer.writerows(full_table_agg)